In [1]:
# 1 Design model (input, output size, forward pass)
# 2 Construct loss and optimizer
# 3 Training loop
#   - Forward pass: compute prediction and loss
#   - Backward pass: gradients
#   - Update weights

In [3]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [4]:
# 0 prepare data

In [5]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

In [6]:
n_samples, n_features = X.shape
print(n_samples, n_features)

569 30


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [9]:
# scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [11]:
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [13]:
# 1. Model
# f = sigmoid(wx +b)

In [14]:
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
    
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred 

In [15]:
model = LogisticRegression(n_features)

In [16]:
# 2 Loss and optimizer

In [23]:
learning_rate = 0.001
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [19]:
# 3. Training loop

In [24]:
num_epochs = 100

for epoch in range(num_epochs):
    # Forward pass and loss
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    # Backward pass
    loss.backward()
    # Update
    optimizer.step()

    # Zero-out accumulated grads ##.loss.backward() will accumulate grads after each epoch
    optimizer.zero_grad()

    if (epoch + 1)%10 == 0:
        print(f"Epoch {epoch + 1}, loss = {loss.item():.4f}")

# do evaluation (i.e. test); no need to keep track of comp graph in this step!
with torch.no_grad():
    y_pred = model(X_test) # return values are sigmoid in (0, 1)
    y_pred_cls = y_pred.round() # W/O torch.no_grad(), this .round() will generate a comp graph
    acc = y_pred_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'Test accuracy = {acc:.4f}')


Epoch 10, loss = 0.1277
Epoch 20, loss = 0.1276
Epoch 30, loss = 0.1275
Epoch 40, loss = 0.1273
Epoch 50, loss = 0.1272
Epoch 60, loss = 0.1271
Epoch 70, loss = 0.1269
Epoch 80, loss = 0.1268
Epoch 90, loss = 0.1267
Epoch 100, loss = 0.1266
Test accuracy = 0.9123
